<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Spring 2021 - Linear-Time Construction of Suffix Trees</h1></center>
<center><h1>Due: Sunday, June 6th, 2021 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter):  Robert Wyatt<br>
    Your student ID (submitter):862134626
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to be documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Ukkonen's algorithm</h1></center>
<br>
In this project you will implement the construction of suffix trees in linear-time using Ukkonen's algorithm. This algorithm was <B>not</B> explained in class. Part of this project is to understand how the algorithm works before starting the implementation. Here is some resources:
<UL>
    <li><A HREF="web.stanford.edu/~mjkay/gusfield.pdf">Gufield's explanation</A> from his <A HREF="https://www.amazon.com/dp/0521585198">book</A></li>
    <li><A HREF="https://www.geeksforgeeks.org/ukkonens-suffix-tree-construction-part-1/">GeeksforGeeks</A></li>
    <li><A HREF="https://stackoverflow.com/questions/9452701/ukkonens-suffix-tree-algorithm-in-plain-english">StackOverflow</A></li>
    <li>There are several videos on YouTube, e.g., <A HREF="https://www.youtube.com/watch?v=aPRqocoBsFQ">this one</A></li>
    <li>Use <A HREF="https://www.google.com/search?q=ukkonen+algorithm">Google</A> for more</li>
</UL>

Goals:
<UL>
    <LI>Write a working (bug-free) Python3 implementation of Ukkonen's algorithm in JupyterHub that runs in linear time</LI>
    <LI>Make sure that your implementation is correct: compare the output of your implementation to the output for the O(n^2) implementation that we used in homework 2</LI>
    <LI>Collect experimental results on running your code for larger and larger inputs, say 1000, 2000, 5000, 10000, 100000 symbols, and plot the running time as a function of the input size</LI>
</UL>

You are allowed to study C/C++ implementations of Ukkonen's algorithm, but you have to write your own Python code. It is mandatory to acknowledge sources.


In [107]:
# YOUR CODE HERE
# no suffix tree libraries allowed

#source: https://www.youtube.com/watch?v=aPRqocoBsFQ Suffix Tree using Ukkonen's algorithm, 
#Tushar Roy - Coding Made Simple

import time
class SuffixTree(object):
    
    def __init__(self, s):
        self.root = self.Node(-2,-2)#-2 on the labels will denote a root node
        ##Time to make our tree using the string s passed in
        length = len(s) 
        activeEdge = -1 #active edge tracks the labS(start label, ie 0,1,2) of edge we are traversing. It is -1 if we are not traversing an edge
        activeLength = 0 #how far along the active edge that we are traversing
        activeNode = self.root #The node that we currently are at
        remaining = 0 #tells us the remaining number of times that the while loop needs to repeate
        
        lastLink = self.root#LastLink is used to know the previous suffix link so that we can correctly link up our nodes
        end = -1 #-1 is a false number used to represent the idea that the node is a leaf. The count of end at any given time is given by endcount
        endCount = -1#tells at any given time what the actual value of a leaf node is. i.e 5,-1 means the leaf node has letters s[i]->s[endcount]
        for i in range(0,length):
            #print("----","s[i] =",s[i],remaining,activeLength)
            remaining = remaining + 1
            endCount = endCount + 1
            lastLink = self.root
            while remaining > 0:
                
                if activeLength == 0:#if active length == 0
                    activeEdge = i#active point change for active lenght equals zero
                   
                if s[activeEdge] not in activeNode.out:#if the current node does not have s[i]
                    #print("new leaf",s[i],i,end)
                    counter = 0
                    
                    activeNode.out[s[i]] = self.Node(i,end)
                    remaining = remaining - 1
                    if lastLink != self.root:
                        lastLink.link = activeNode
                        lastLink == self.root
                    activeNode = activeNode.link#seems to fix it
                    
                    
                else:          
                    tempEndConverter = 0
                    if activeNode.out[s[activeEdge]].labE == -1:
                        tempEndConverter= endCount
                    else:
                        tempEndConverter = activeNode.out[s[activeEdge]].labE
                        
                    if tempEndConverter - activeNode.out[s[activeEdge]].labS + 1 <= activeLength:
                        #print("**past an edge test.","labE =",tempEndConverter,",labS =",activeNode.out[s[activeEdge]].labS,",activeLenght =",activeLength,",s[activeEdge] = ",s[activeEdge],"remaining = ", remaining,"activeEdge = ",activeEdge)
                        activeNode = activeNode.out[s[activeEdge]]
                        activeEdge = activeEdge + (tempEndConverter - activeNode.labS+1)
                        activeLength = activeLength - (tempEndConverter - activeNode.labS+1)
                        #activeNode = activeNode.out[s[activeEdge]]
                        continue
                   
                    if  s[activeLength + activeNode.out[s[activeEdge]].labS] == s[i]: #activates if the edge that we are traversing has s[i]
                        #print("short stop","s[labs + activeLenght] =",s[activeNode.out[s[activeEdge]].labS + activeLength],",labs + activeLenght = ",activeNode.out[s[activeEdge]].labS + activeLength,"activeLenght = ",activeLength)
                        if lastLink != self.root and activeNode != self.root:
                            lastLink.link = activeNode
                            lastLink = self.root
                        activeLength = activeLength + 1
                        break

                    #newP = self.Node(activeNode.out[s[activeEdge]].labS,activeNode.out[s[activeEdge]].labS + activeLength-1)#new parent, goes from labS(start of origional edge)->labS+activelength-1
                    newP = self.Node(activeNode.out[s[activeEdge]].labS, activeNode.out[s[activeEdge]].labS + activeLength-1)
                    #c1 = self.Node(activeNode.out[s[activeEdge]].labS + activeLength, activeNode.out[s[activeEdge]].labE)#non-leaf child, goes from labS+activelength->labE(end of og edge)
                    c1 = self.Node(activeNode.out[s[activeEdge]].labS + activeLength, activeNode.out[s[activeEdge]].labE)
                    c1.out = activeNode.out[s[activeEdge]].out
                    c1.link = activeNode.out[s[activeEdge]].link
                    c2 = self.Node(i,end)#leaf child, goes from i->end
                    
                    #print("activeLenght is > 0 test, node splits into","newP:",activeNode.out[s[activeEdge]].labS,activeNode.out[s[activeEdge]].labS+
                                 #activeLength-1,"c1:",activeNode.out[s[activeEdge]].labS + activeLength,activeNode.out[s[activeEdge]].labE,"c2:",i,end,
                                 #"activeLength =",activeLength)
                            

                    newP.out[s[c1.labS]] = c1#these two lines gives the new parent it's 2 children
                    newP.out[s[c2.labS]] = c2
                    activeNode.out[s[activeEdge]] = newP#overwrites the old edge with the new parent
                    newP.link = self.root#sets newP's link to the root
                    if lastLink != self.root: 
                        lastLink.link = newP # if the lastLink was not the root 
                    lastLink = newP
                    remaining = remaining - 1
                    if activeNode != self.root:
                        activeNode = activeNode.link
                    else:
                        activeLength = activeLength-1
                        activeEdge = i-remaining +1
                        


    def longestRepeat(self,inputString):
        global longest
        longest = ''
        self.root.longestRepeat('',0,inputString)
        return longest
    
    def traverse(self,s):
        return self.root.traverse(0,s)  
                
    class Node(object):
        def __init__(self, labS, labE):
            self.labS = labS # label on path leading to this node
            self.labE = labE
            self.out = {}  # outgoing edges; maps characters to nodes
            self.link = self # makes a suffix link; sets it to itself
            
        def traverse(self, level,s):
            for x in self.out:          # x is the first letter of the outgoing edge, if self.out=={} we are in a leaf and the for loop is not executed
                child = self.out[x]     # out[x] gives the pointer to the children with an edge that starts with x
                #print("incoming label:",x)
                #print(level,child.labS,child.labE)
                print("-------")
                if child.labS == child.labE:
                    print(level,s[child.labS])
                elif child.labE == -1:
                    print(level,s[child.labS:len(s)])
                else:
                    print(level,s[child.labS:child.labE+1],child.labS,child.labE)
                    #print(level,s[child.labS:child.labE],child.labS,child.labE)
                child.traverse(level+1,s) # visit recursively the child
                
        def longestRepeat(self,s,level,inputString):
            """ Returns the longest repeated string by finding the internal node in the suffix
                tree that corresponds to the longest string from the root"""
            global longest
            level += 1
            if len(s) > len(longest):
                longest = s
            for x in self.out:          # x is the first letter of the outgoing edge, if self.out=={} we are in a leaf and the for loop is not executed
                
                child = self.out[x]     # out[x] gives the pointer to the children with an edge that starts with x
                if level == 1:
                    child.longestRepeat('',level+1,inputString)# visit recursively the child
                else:
                    child.longestRepeat(s+inputString[self.labS:self.labE+1],level+1,inputString)
            

print("began")
start = time.time()
#reader = open("1000 word tree.txt","r")
#reader = open("2000 word tree.txt","r")
#reader = open("5000 word tree.txt","r")
#reader = open("10000 word tree.txt","r")
reader = open("100000 word tree.txt","r")
stree = SuffixTree(reader.readline())
#instring = reader.readline()+"$"
#stree = SuffixTree(instring)



print("finished",time.time()-start)


began
finished 0.782477617263794


In [103]:
stree = SuffixTree("AAAAABBBB$")
stree.longestRepeat("AAAAABBBB$")

'AAAA'

In [104]:
stree = SuffixTree("xybxycxyb$")
stree.longestRepeat("xybxycxyb$")

'xyb'

1000 Word
began Wed Jun  2 09:25:06 2021
finished Wed Jun  2 09:25:06 2021

In [72]:
stree = SuffixTree("AAAAAAAAA$")
stree.longestRepeat("AAAAAAAAA$")

'AAAAAAAA'

In [73]:
stree = SuffixTree("GTCATCACGGTAGGGAAAGAGTTCTTAGTTGTTGCCAGCCCGGCCACAATCCGATACCCCTCAGCCATTCTATGGCAACTTATTCTATGTTGGTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAACCTTCACGCCAGACCAGAGATTACCTTGAAAATGGCCTAAGGTCGCCCGCCACCCGACGGATTGAACGTTTCCGTGCTACATTCCATGGTATGCGGTAATCAGGATAACTCACCCACGTCGAATAGCATGCCTATGAGCGGTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAACGCCTAATCTTCTTGGAAAACTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAACCCGTGTAGAGATGGCGGACTTATGTTACGCCTCTGTGAACAATATAGAGGTCCACGGCGCGTGCGATTGGACCTTGGAACTACCATAACGAAAGGACCATGATAGCTGAAGGCCGCACCTCGCGACGGAAACAACGCCTATCTGATCATGCTGGGTGATAATCGCGAAGACCAGTTCGAAGGGTTTTCTTCCAACGGTCTTAAGGACTTCCCTAGCCTGAAGGGTGGCGGCCCTATTTCGCCTATGGGTCCTGAGGGCTCGGGCTTTAGTCGTCGAAAGCAGGTATAGACAGGATCCACACACAACTAGGGTGACTTATTCGTTACGCCAGCGTGCTCTCGCCTGATATGATAATTAGGTAGTAGGTGTAATCCCGGTTGCTCAAGGGCCCATCATCATGCCTTTCCTCTGCGGAGACTCCTATTCTATATTCTTTACTGGCTTTTCATAGTCATGAGGAGTCCGTACAGGAATTGGATTGTCAGCTAAACGAGTTAGGACGCAGTTCCAATTATGTCATATCCGGACTCACACAAATGCCCAAGAGCGTTAAGGAAAGTCTACTGGTGTAGGCCACGCTTATACGTTATTGAATGGATGGGTCAATATTCGAGTACCATCGGCCCCTTAAACATTACCTATAGTATGATAAACGCGGCTATCTCTAGAACAGAGTCTGCATGCCCATGTGTAGGTTTAACGGATAAATAAAACGCGACGTGCAAGGGCCGACTTCAATTGCAATGC$")
#GOAAL:GGTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAAC
#       

stree.longestRepeat("GTCATCACGGTAGGGAAAGAGTTCTTAGTTGTTGCCAGCCCGGCCACAATCCGATACCCCTCAGCCATTCTATGGCAACTTATTCTATGTTGGTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAACCTTCACGCCAGACCAGAGATTACCTTGAAAATGGCCTAAGGTCGCCCGCCACCCGACGGATTGAACGTTTCCGTGCTACATTCCATGGTATGCGGTAATCAGGATAACTCACCCACGTCGAATAGCATGCCTATGAGCGGTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAACGCCTAATCTTCTTGGAAAACTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAACCCGTGTAGAGATGGCGGACTTATGTTACGCCTCTGTGAACAATATAGAGGTCCACGGCGCGTGCGATTGGACCTTGGAACTACCATAACGAAAGGACCATGATAGCTGAAGGCCGCACCTCGCGACGGAAACAACGCCTATCTGATCATGCTGGGTGATAATCGCGAAGACCAGTTCGAAGGGTTTTCTTCCAACGGTCTTAAGGACTTCCCTAGCCTGAAGGGTGGCGGCCCTATTTCGCCTATGGGTCCTGAGGGCTCGGGCTTTAGTCGTCGAAAGCAGGTATAGACAGGATCCACACACAACTAGGGTGACTTATTCGTTACGCCAGCGTGCTCTCGCCTGATATGATAATTAGGTAGTAGGTGTAATCCCGGTTGCTCAAGGGCCCATCATCATGCCTTTCCTCTGCGGAGACTCCTATTCTATATTCTTTACTGGCTTTTCATAGTCATGAGGAGTCCGTACAGGAATTGGATTGTCAGCTAAACGAGTTAGGACGCAGTTCCAATTATGTCATATCCGGACTCACACAAATGCCCAAGAGCGTTAAGGAAAGTCTACTGGTGTAGGCCACGCTTATACGTTATTGAATGGATGGGTCAATATTCGAGTACCATCGGCCCCTTAAACATTACCTATAGTATGATAAACGCGGCTATCTCTAGAACAGAGTCTGCATGCCCATGTGTAGGTTTAACGGATAAATAAAACGCGACGTGCAAGGGCCGACTTCAATTGCAATGC$")


'GTGGGATTTTCTGCCGGCATCGACCACTAGACAGCGTTCTCGGGCTGCAGCCAATTCCATTAAC'

In [74]:
stree = SuffixTree("AATTTCCGACTTGCATGACGAGTCAGCGTTCCATCTGATCGAGTCTCCGAAGAACAAATACCCCTACTCAGTTGTGAGCCCCTTTACCGTGAGGACAGGGTCCTTGATGTCGTCTCCTAATTTGCGTTGCGGCTCAACATGTTGTACATAGTGGGGCCAGCCCCAGGGATTTTGTAATTTCTACACTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAACTTAATGCCATTCCTACATCCTCGATACCTCGTCTGTCAGAGCAATGAGCTGGTTAGAGGACAGTATTGGTCGGTCATCCTCAGATTGGGGACACATCCGTCTCTATGTGCGTTCCGTTGCCTTGTGCTGACCTTGTCGAACGTACCCCATCTTCGAGCCGCACGCTCGACCAGCTAGGTCCCAGCAGTGGCCTGATAGAAAAATTACCTACGGGCCTCCCAATCGTCCTCCCAGGGTGTCGAACTCTCAAAATTCCCGCATGGTCGTGCTTCCGTACGAATTATGCAAACTCCAGAACCCGGATCTATTCCACGCTCAACGAGTCCTTCACGCTTGGTAGAATTTCATGCTCGTCTTTTGTATCCGTGTAAGTAGGAGGCCGCTGTACGGGTATCCCAGCCTTCGCGCTCTGCTGCAGGGACGTTAACACTCCGAACTTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAGCTCTGAGGGATCCTCACGGAGTTAGATTTATTTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAGAGTGATGTTTGGAATGCCAACTTCCATGCACGCCAATTGAGCAATCAGGAGAATCGAGTGCTGTTGACCTAGACCTTGTCAGAAGTATGAATTAACCGCGCGTGTAGGTTTGTCGCTCGACCTGCAAGGGTGCACAATCTGGACTGTCGTCGGCGAACGCTTTCATACGCCTACAAACCGCGTTGCTGGTCGAATCGATCTCACCACCGGCCTTGCAGGATTCTAATTATTCTCTCTCGGTGAGACTGCCGGCGGTCCATGGGTCTGTGTTTCGCTTCAAGCAGTGATATACTGGCGTTTTGTGACACATGGCCACGCACGCCTCTCGTTACTCCCAAT$")
#GOAL: TTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAG
#       
stree.longestRepeat("AATTTCCGACTTGCATGACGAGTCAGCGTTCCATCTGATCGAGTCTCCGAAGAACAAATACCCCTACTCAGTTGTGAGCCCCTTTACCGTGAGGACAGGGTCCTTGATGTCGTCTCCTAATTTGCGTTGCGGCTCAACATGTTGTACATAGTGGGGCCAGCCCCAGGGATTTTGTAATTTCTACACTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAACTTAATGCCATTCCTACATCCTCGATACCTCGTCTGTCAGAGCAATGAGCTGGTTAGAGGACAGTATTGGTCGGTCATCCTCAGATTGGGGACACATCCGTCTCTATGTGCGTTCCGTTGCCTTGTGCTGACCTTGTCGAACGTACCCCATCTTCGAGCCGCACGCTCGACCAGCTAGGTCCCAGCAGTGGCCTGATAGAAAAATTACCTACGGGCCTCCCAATCGTCCTCCCAGGGTGTCGAACTCTCAAAATTCCCGCATGGTCGTGCTTCCGTACGAATTATGCAAACTCCAGAACCCGGATCTATTCCACGCTCAACGAGTCCTTCACGCTTGGTAGAATTTCATGCTCGTCTTTTGTATCCGTGTAAGTAGGAGGCCGCTGTACGGGTATCCCAGCCTTCGCGCTCTGCTGCAGGGACGTTAACACTCCGAACTTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAGCTCTGAGGGATCCTCACGGAGTTAGATTTATTTTCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAAGAGTGATGTTTGGAATGCCAACTTCCATGCACGCCAATTGAGCAATCAGGAGAATCGAGTGCTGTTGACCTAGACCTTGTCAGAAGTATGAATTAACCGCGCGTGTAGGTTTGTCGCTCGACCTGCAAGGGTGCACAATCTGGACTGTCGTCGGCGAACGCTTTCATACGCCTACAAACCGCGTTGCTGGTCGAATCGATCTCACCACCGGCCTTGCAGGATTCTAATTATTCTCTCTCGGTGAGACTGCCGGCGGTCCATGGGTCTGTGTTTCGCTTCAAGCAGTGATATACTGGCGTTTTGTGACACATGGCCACGCACGCCTCTCGTTACTCCCAAT$")


'TCCATATACGGGACAAGGGTGAGCATTTCCGGGCTTGGATAGGGGCTGCAAGAAAATATCTGGACGTAAGAA'